In [1]:
import requests
import json

In [20]:
API_URL = "https://stream.wikimedia.org/v2/stream/recentchange"
response = requests.get(API_URL, stream=True,timeout=10)

count = 0 
for line in response.iter_lines():
    decoded_line = line.decode('utf-8')  # Decode bytes to string
    if decoded_line.startswith("data:"):
        json_data = decoded_line[5:].strip()
        json_data = json.loads(json_data)
        print(json_data)

        print(json_data.keys())

    # decoded_line = json.loads(line)
    # print(decoded_line)
    count +=1
    if count == 50:
        break
    
print(decoded_line)

{'$schema': '/mediawiki/recentchange/1.0.0', 'meta': {'uri': 'https://www.wikidata.org/wiki/Q496115', 'request_id': '34ffaa37-37cf-43df-9a99-8dc3df96bb0c', 'id': '1a90285f-8568-4f89-b884-a333903885bc', 'dt': '2024-11-15T03:06:28Z', 'domain': 'www.wikidata.org', 'stream': 'mediawiki.recentchange', 'topic': 'codfw.mediawiki.recentchange', 'partition': 0, 'offset': 1248717693}, 'id': 2342899586, 'type': 'edit', 'namespace': 0, 'title': 'Q496115', 'title_url': 'https://www.wikidata.org/wiki/Q496115', 'comment': '/* wbsetaliases-add:1|sl */ Okrožje Phelps', 'timestamp': 1731639988, 'user': 'TadejM', 'bot': False, 'notify_url': 'https://www.wikidata.org/w/index.php?diff=2274892762&oldid=2274892760&rcid=2342899586', 'minor': False, 'patrolled': True, 'length': {'old': 65452, 'new': 65536}, 'revision': {'old': 2274892760, 'new': 2274892762}, 'server_url': 'https://www.wikidata.org', 'server_name': 'www.wikidata.org', 'server_script_path': '/w', 'wiki': 'wikidatawiki', 'parsedcomment': '\u200e<

In [28]:
import requests
import json
import time
from confluent_kafka import Producer
from requests.exceptions import ChunkedEncodingError

API_URL = "https://stream.wikimedia.org/v2/stream/recentchange"
KAFKA_TOPIC = "your_kafka_topic"

conf = {
    'bootstrap.servers': 'localhost:9092',
    'client.id': 'your_client_id'
}

producer = Producer(conf)

def delivery_report(err, msg):
    if err:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [partition {msg.partition()}] at offset {msg.offset()}")

def connect_and_stream(max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(API_URL, stream=True, timeout=10)
            
            if response.status_code == 200:
                print("Successfully connected to the stream!")
                
                for line in response.iter_lines():
                    if line:
                        decoded_line = line.decode('utf-8').strip()

                        # Process only lines that start with "data:"
                        if decoded_line.startswith("data:"):
                            json_data = decoded_line[5:].strip()

                            try:
                                event_data = json.loads(json_data)
                                print("Parsed Event Data:", event_data)

                                producer.produce(
                                    KAFKA_TOPIC,
                                    value=json.dumps(event_data),
                                    callback=delivery_report
                                )
                                producer.poll(0)
                                
                            except json.JSONDecodeError as e:
                                print(f"Error decoding JSON: {e} - Line: {json_data}")

                producer.flush()
            else:
                print(f"Failed to connect to API. Status code: {response.status_code}")

        except ChunkedEncodingError:
            print("Connection lost. Retrying...")
            retries += 1
            time.sleep(5)  # Wait before retrying

        except Exception as e:
            print(f"Unexpected error: {e}")
            retries += 1
            time.sleep(5)  # Wait before retrying

    print("Max retries reached. Exiting.")
    producer.flush()  # Final flush on exit

# Start streaming with retry logic
connect_and_stream()


%4|1731235177.515|TERMINATE|your_client_id#producer-12| [thrd:app]: Producer terminating with 2 messages (2482 bytes) still in queue or transit: use flush() to wait for outstanding message delivery


Successfully connected to the stream!
Parsed Event Data: {'$schema': '/mediawiki/recentchange/1.0.0', 'meta': {'uri': 'https://commons.wikimedia.org/wiki/File:%D0%A2%D1%80%D0%B5%D0%BD%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B0_%D0%BF%D0%BE_%D0%B4%D0%B5%D1%81%D0%B0%D0%BD%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8E_%D0%B8%D0%B7_%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D0%BB%D0%B5%D1%82%D0%BE%D0%B2_%D0%B2%D0%BE%D0%B5%D0%BD%D0%BD%D0%BE%D1%81%D0%BB%D1%83%D0%B6%D0%B0%D1%89%D0%B8%D1%85_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8_%D0%B8_%D0%9F%D0%B0%D0%BA%D0%B8%D1%81%D1%82%D0%B0%D0%BD%D0%B0_%D0%BD%D0%B0_%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B8_%C2%AB%D0%94%D1%80%D1%83%D0%B6%D0%B1%D0%B0-2016%C2%BB_(4).jpg', 'request_id': 'befa08a4-3546-4d08-a359-9c5fa1a657dc', 'id': '8bfe1701-5490-4d5c-a740-7d3a814cafc2', 'dt': '2024-11-10T10:39:38Z', 'domain': 'commons.wikimedia.org', 'stream': 'mediawiki.recentchange', 'topic': 'codfw.mediawiki.recentchange', 'partition': 0, 'offset': 1237881225}, 'id': 2651303583, 'type': 'ed

KeyboardInterrupt: 

In [5]:
import requests
import json
import time
from confluent_kafka import Producer
from requests.exceptions import ChunkedEncodingError

API_URL = "https://stream.wikimedia.org/v2/stream/recentchange"
KAFKA_TOPIC = "your_kafka_topic"

conf = {
    'bootstrap.servers': 'localhost:9092',
    'client.id': 'your_client_id'
}

producer = Producer(conf)

def delivery_report(err, msg):
    if err:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [partition {msg.partition()}] at offset {msg.offset()}")

def connect_and_stream(max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(API_URL, stream=True, timeout=10)
            
            if response.status_code == 200:
                print("Successfully connected to the stream!")
                
                for line in response.iter_lines():
                    # if line:
                    #     decoded_line = line.decode('utf-8').strip()
                    # Assuming line is in bytes format
                    decoded_line = line.decode('utf-8')  # Decode bytes to string

                        # Process only lines that start with "data:"
                        # if decoded_line.startswith("data:"):
                        #     json_data = decoded_line[5:].strip()

                    try:
                        # event_data = json.loads(json_data)
                        # print("Parsed Event Data:", event_data)

                        producer.produce(
                            KAFKA_TOPIC,
                            value=json.dumps(decoded_line),
                            callback=delivery_report
                        )
                        producer.poll(0)
                        
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON: {e} - Line: {line}")

                producer.flush()
            else:
                print(f"Failed to connect to API. Status code: {response.status_code}")

        except ChunkedEncodingError:
            print("Connection lost. Retrying...")
            retries += 1
            time.sleep(5)  # Wait before retrying

        except Exception as e:
            print(f"Unexpected error: {e}")
            retries += 1
            time.sleep(5)  # Wait before retrying

    print("Max retries reached. Exiting.")
    producer.flush()  # Final flush on exit

# Start streaming with retry logic
connect_and_stream()


Successfully connected to the stream!
Message delivered to your_kafka_topic [partition 0] at offset 80216
Message delivered to your_kafka_topic [partition 0] at offset 80217
Message delivered to your_kafka_topic [partition 0] at offset 80218
Message delivered to your_kafka_topic [partition 0] at offset 80219
Message delivered to your_kafka_topic [partition 0] at offset 80220
Message delivered to your_kafka_topic [partition 0] at offset 80221
Message delivered to your_kafka_topic [partition 0] at offset 80222
Message delivered to your_kafka_topic [partition 0] at offset 80223
Message delivered to your_kafka_topic [partition 0] at offset 80224
Message delivered to your_kafka_topic [partition 0] at offset 80225
Message delivered to your_kafka_topic [partition 0] at offset 80226
Message delivered to your_kafka_topic [partition 0] at offset 80227
Message delivered to your_kafka_topic [partition 0] at offset 80228
Message delivered to your_kafka_topic [partition 0] at offset 80229
Message de

KeyboardInterrupt: 

In [ ]:
## batch processing of api

batch_size = 100  # Adjust based on your use case
messages = []

# Collect a batch of messages
for line in response.iter_lines():
    if line:
        decoded_line = line.decode('utf-8').strip()
        messages.append(decoded_line)
        if len(messages) >= batch_size:
            # Produce the batch to Kafka
            for msg in messages:
                producer.produce(KAFKA_TOPIC, value=msg, callback=delivery_report)
            # Reset batch
            messages = []
            producer.poll(0)  # Handle delivery reports

# Send any remaining messages
for msg in messages:
    producer.produce(KAFKA_TOPIC, value=msg, callback=delivery_report)
producer.poll(0)  # Final poll to flush any outstanding messages
